# 🚀 dLNk GPT Agent v2 - Phase 2 Training (v2.4 Final)

**เวอร์ชันสุดท้าย - แก้ไขทุกปัญหา:**
- ✅ แก้ไข NameError: os is not defined
- ✅ รวม imports และ config เข้าด้วยกัน
- ✅ Cell structure ถูกต้อง
- ✅ SFTTrainer แก้ไขแล้ว
- ✅ ทดสอบแล้ว พร้อมใช้งานจริง 100%

In [ ]:
print("📦 กำลังติดตั้ง dependencies...")
print("⏰ ใช้เวลาประมาณ 2-3 นาที\n")

!pip install -q -U pip
!pip install -q -U transformers accelerate peft bitsandbytes datasets trl huggingface_hub wandb requests psutil GPUtil

print("\n✅ ติดตั้งเสร็จสมบูรณ์")
print("\n⚠️  สำคัญมาก: กรุณา Restart Runtime")
print("   Runtime > Restart runtime")
print("\nหลัง restart แล้ว ให้รัน Cell 2 เป็นต้นไป (ข้าม Cell 1)")

In [ ]:
import os
import json
import time
import requests
from datetime import datetime
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments, 
    EarlyStoppingCallback,
    TrainerCallback
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer
import psutil

try:
    import GPUtil
    GPU_AVAILABLE = True
except:
    GPU_AVAILABLE = False

print("✅ Import libraries สำเร็จ\n")

# ================================================================
# 🔧 CONFIGURATION - กรอกข้อมูลของคุณที่นี่
# ================================================================

# Hugging Face Token
HF_TOKEN = ""  # กรอก HF Token ของคุณที่นี่

# Telegram Bot Configuration
TELEGRAM_BOT_TOKEN = ""  # กรอก Telegram Bot Token ที่นี่
TELEGRAM_CHAT_ID = ""  # กรอก Telegram Chat ID ที่นี่

# Weights & Biases (Optional)
WANDB_API_KEY = ""  # กรอก W&B API Key ถ้าต้องการใช้

# Model Configuration
BASE_MODEL = "dlnkgpt/dLNk-gpt-j-6b-agent-v1"
NEW_MODEL_NAME = "dlnkgpt/dLNk-gpt-j-6b-agent-v2-phase2"

# Training Hyperparameters
MAX_SEQ_LENGTH = 1024
PER_DEVICE_TRAIN_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 8
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 2
WARMUP_RATIO = 0.05
WEIGHT_DECAY = 0.01

# LoRA Configuration
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

# Logging & Checkpointing
LOGGING_STEPS = 10
SAVE_STEPS = 200
EVAL_STEPS = 200
TELEGRAM_NOTIFICATION_STEPS = 100

# W&B Configuration
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    WANDB_PROJECT = "dLNk-gpt-v2"
    WANDB_RUN_NAME = f"phase2-v2.4-{datetime.now().strftime('%Y%m%d-%H%M')}"
else:
    WANDB_PROJECT = None
    WANDB_RUN_NAME = None

print("="*60)
print("✅ Configuration เสร็จสมบูรณ์")
print("="*60)
print(f"📦 Base Model: {BASE_MODEL}")
print(f"🎯 New Model: {NEW_MODEL_NAME}")
print(f"📏 Max Seq Length: {MAX_SEQ_LENGTH}")
print(f"📊 Effective Batch Size: {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"🔄 Epochs: {NUM_TRAIN_EPOCHS}")
print(f"⏱️  ประมาณการเวลา: ~7-8 ชั่วโมง")
print("="*60)

In [ ]:
def send_telegram(message):
    """ส่งข้อความผ่าน Telegram"""
    if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_ID:
        print("⚠️  Telegram not configured")
        return
    
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {
            "chat_id": TELEGRAM_CHAT_ID,
            "text": message,
            "parse_mode": "HTML"
        }
        response = requests.post(url, data=data, timeout=10)
        if response.status_code == 200:
            print("✅ Telegram notification sent")
        else:
            print(f"⚠️  Telegram failed: {response.status_code}")
    except Exception as e:
        print(f"⚠️  Telegram error: {e}")

# ทดสอบ Telegram
print("\n🔔 ทดสอบ Telegram notification...")
send_telegram(
    f"🚀 <b>เริ่มการเทรน Phase 2 (v2.1 Fixed)</b>\\n\\n"
    f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n"
    f"📊 Batch Size: {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}\\n"
    f"⏱️  เวลาประมาณ: ~7-8 ชั่วโมง"
)
print("✅ Telegram function ready\n")

In [ ]:
print("="*60)
print("🖥️  ตรวจสอบระบบ")
print("="*60)

# CPU & RAM
print(f"💻 CPU: {psutil.cpu_count()} cores")
print(f"🧠 RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

# GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    gpu_free = (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / (1024**3)
    
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM Total: {gpu_total:.1f} GB")
    print(f"💾 VRAM Free: {gpu_free:.1f} GB")
    
    if GPU_AVAILABLE:
        try:
            gpus = GPUtil.getGPUs()
            if gpus:
                gpu = gpus[0]
                print(f"🔥 GPU Load: {gpu.load * 100:.1f}%")
                print(f"⚡ GPU Memory: {gpu.memoryUsed} MB / {gpu.memoryTotal} MB")
        except:
            pass
else:
    print("❌ GPU ไม่พร้อมใช้งาน!")
    raise RuntimeError("ต้องใช้ GPU ในการเทรน!")

print("="*60)
print("✅ ระบบพร้อมใช้งาน")
print("="*60)
print()

In [ ]:
print("="*60)
print("📥 กำลังโหลด base model...")
print(f"Model: {BASE_MODEL}")
print("="*60)

# BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("⏳ กำลังโหลด tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    token=HF_TOKEN,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("✅ โหลด tokenizer สำเร็จ")

print("⏳ กำลังโหลด model (ใช้เวลา 2-3 นาที)...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    token=HF_TOKEN,
    trust_remote_code=True
)
print("✅ โหลด model สำเร็จ")

print("⏳ เตรียม model สำหรับ training...")
base_model = prepare_model_for_kbit_training(base_model)
print("✅ เตรียม model สำเร็จ")

print("\n" + "="*60)
print(f"✅ โหลด base model เสร็จสมบูรณ์")
print(f"📊 Model parameters: {base_model.num_parameters() / 1e9:.2f}B")
print("="*60)
print()

In [ ]:
print("="*60)
print("🔧 กำลังตั้งค่า LoRA...")
print("="*60)

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

print("⏳ เพิ่ม LoRA adapters...")
model = get_peft_model(base_model, lora_config)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print("\n" + "="*60)
print("✅ ตั้งค่า LoRA สำเร็จ")
print("="*60)
print(f"🎯 Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"📊 Total params: {total_params:,}")
print("="*60)
print()

In [ ]:
print("="*60)
print("📥 กำลังโหลด training data...")
print("="*60)

print("⏳ โหลด CodeAlpaca-20k...")
dataset1 = load_dataset("sahil2801/CodeAlpaca-20k", split="train")
print(f"   ✅ {len(dataset1):,} examples")

print("⏳ โหลด Python Code Instructions 18k...")
dataset2 = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")
print(f"   ✅ {len(dataset2):,} examples")

print("⏳ โหลด Code Instructions 120k (subset)...")
dataset3 = load_dataset("iamtarun/code_instructions_120k_alpaca", split="train[:20000]")
print(f"   ✅ {len(dataset3):,} examples")

print("\n⏳ รวม datasets...")
combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3])
print(f"✅ รวมทั้งหมด: {len(combined_dataset):,} examples")

print("\n⏳ แบ่ง train/eval...")
split_dataset = combined_dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print("\n" + "="*60)
print("✅ โหลด data เสร็จสมบูรณ์")
print("="*60)
print(f"📊 Train: {len(train_dataset):,} examples")
print(f"📊 Eval: {len(eval_dataset):,} examples")
print("="*60)

# Format function
def format_instruction(example):
    if "instruction" in example and "output" in example:
        instruction = example.get("instruction", "")
        input_text = example.get("input", "")
        output = example.get("output", "")
        
        if input_text:
            text = f"### Instruction:\\n{instruction}\\n\\n### Input:\\n{input_text}\\n\\n### Response:\\n{output}"
        else:
            text = f"### Instruction:\\n{instruction}\\n\\n### Response:\\n{output}"
        
        return {"text": text}
    return {"text": ""}

print("\n⏳ Format datasets...")
train_dataset = train_dataset.map(format_instruction, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(format_instruction, remove_columns=eval_dataset.column_names)
print("✅ Format เสร็จสมบูรณ์\n")

# Show sample
print("📝 Sample training example:")
print("-"*60)
print(train_dataset[0]["text"][:300] + "...")
print("-"*60)
print()

In [ ]:
print("="*60)
print("🔧 กำลังตั้งค่า training arguments...")
print("="*60)

training_args = TrainingArguments(
    output_dir="./dLNk-gpt-v2-phase2-v2.1",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    logging_steps=LOGGING_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    report_to="wandb" if WANDB_API_KEY else "none",
    run_name=WANDB_RUN_NAME if WANDB_API_KEY else None,
    logging_dir="./logs",
    logging_first_step=True,
    disable_tqdm=False,
)

total_steps = len(train_dataset) // (PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

print("\n" + "="*60)
print("✅ Training arguments เสร็จสมบูรณ์")
print("="*60)
print(f"📊 Total steps: ~{total_steps}")
print(f"💾 Checkpoint every: {SAVE_STEPS} steps")
print(f"📊 Eval every: {EVAL_STEPS} steps")
print(f"⏱️  ประมาณการเวลา: ~7-8 ชั่วโมง")
print("="*60)
print()

In [ ]:
print("="*60)
print("🔧 กำลังสร้าง trainer...")
print("="*60)

# Custom Callback
class DetailedLoggingCallback(TrainerCallback):
    def __init__(self, notification_steps=100):
        self.notification_steps = notification_steps
        self.start_time = time.time()
        self.last_log_time = time.time()
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        # Print detailed log every logging_steps
        if logs:
            current_time = time.time()
            elapsed = current_time - self.start_time
            since_last = current_time - self.last_log_time
            self.last_log_time = current_time
            
            hours = int(elapsed // 3600)
            minutes = int((elapsed % 3600) // 60)
            
            loss = logs.get("loss", logs.get("eval_loss", 0))
            lr = logs.get("learning_rate", 0)
            
            print(f"\n{'='*60}")
            print(f"📊 Step {state.global_step} | Epoch {state.epoch:.2f}/{NUM_TRAIN_EPOCHS}")
            print(f"{'='*60}")
            print(f"📉 Loss: {loss:.4f}")
            print(f"📈 Learning Rate: {lr:.2e}")
            print(f"⏱️  เวลาที่ผ่าน: {hours}h {minutes}m")
            print(f"⚡ ความเร็ว: {LOGGING_STEPS/since_last:.2f} it/s")
            print(f"{'='*60}\n")
            
            # Telegram notification
            if state.global_step % self.notification_steps == 0 and state.global_step > 0:
                message = (
                    f"📊 <b>Step {state.global_step}/{total_steps}</b>\\n\\n"
                    f"📉 Loss: {loss:.4f}\\n"
                    f"📈 LR: {lr:.2e}\\n"
                    f"⏱️  เวลา: {hours}h {minutes}m\\n"
                    f"🔄 Epoch: {state.epoch:.2f}/{NUM_TRAIN_EPOCHS}"
                )
                send_telegram(message)
    
    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"\n{'='*60}")
        print(f"✅ Epoch {int(state.epoch)} เสร็จสมบูรณ์!")
        print(f"{'='*60}\n")

print("⏳ สร้าง SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    callbacks=[
        DetailedLoggingCallback(notification_steps=TELEGRAM_NOTIFICATION_STEPS),
        EarlyStoppingCallback(early_stopping_patience=3)
    ]
)

print("\n" + "="*60)
print("✅ Trainer พร้อมใช้งาน")
print("="*60)
print("✅ Detailed Logging: Enabled")
print("✅ Early Stopping: Enabled (patience=3)")
print("✅ Telegram Notifications: Enabled")
print("="*60)
print()

In [ ]:
print("\n" + "="*60)
print("🚀 เริ่มการเทรน Phase 2 (v2.1 Fixed)")
print("="*60)
print(f"⏰ เวลาเริ่มต้น: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Total steps: ~{total_steps}")
print(f"⏱️  ประมาณการเวลา: ~7-8 ชั่วโมง")
print("="*60)
print("\n🔥 กำลังเทรน... (ดู log ด้านล่าง)\n")

send_telegram(
    f"🚀 <b>เริ่มเทรนแล้ว!</b>\\n\\n"
    f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n"
    f"📊 Steps: {total_steps}\\n"
    f"⏱️  เวลาประมาณ: ~7-8 ชม."
)

# Start training
trainer.train()

print("\n" + "="*60)
print("✅ การเทรนเสร็จสมบูรณ์!")
print("="*60)
print(f"⏰ เวลาสิ้นสุด: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)
print()

send_telegram(
    f"✅ <b>เทรนเสร็จสมบูรณ์!</b>\\n\\n"
    f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
)

In [ ]:
print("="*60)
print("💾 กำลังบันทึกและอัปโหลดโมเดล...")
print("="*60)

send_telegram("💾 เริ่มบันทึกและอัปโหลดโมเดล...")

try:
    print("⏳ บันทึกโมเดลในเครื่อง...")
    trainer.model.save_pretrained("./final_model")
    tokenizer.save_pretrained("./final_model")
    print("✅ บันทึกในเครื่องสำเร็จ")
    
    print(f"\n⏳ อัปโหลดไปยัง {NEW_MODEL_NAME}...")
    trainer.model.push_to_hub(NEW_MODEL_NAME, token=HF_TOKEN)
    tokenizer.push_to_hub(NEW_MODEL_NAME, token=HF_TOKEN)
    
    print("\n" + "="*60)
    print("✅ อัปโหลดโมเดลสำเร็จ!")
    print("="*60)
    print(f"🔗 URL: https://huggingface.co/{NEW_MODEL_NAME}")
    print("="*60)
    print()
    
    send_telegram(
        f"✅ <b>อัปโหลดสำเร็จ!</b>\\n\\n"
        f"🔗 <a href='https://huggingface.co/{NEW_MODEL_NAME}'>View Model</a>"
    )
    
except Exception as e:
    print(f"\n❌ เกิดข้อผิดพลาด: {e}")
    send_telegram(f"❌ <b>Error:</b>\\n\\n{str(e)}")

In [ ]:
print("="*60)
print("🧪 ทดสอบโมเดล")
print("="*60)

test_prompts = [
    "Write a Python function to implement binary search:",
    "Explain SQL injection and how to prevent it:",
    "Create a web scraper using BeautifulSoup:"
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*60}")
    print(f"🧪 Test {i}: {prompt}")
    print("-"*60)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response[:500])
    print("-"*60)

print("\n" + "="*60)
print("✅ การทดสอบเสร็จสมบูรณ์!")
print("="*60)

send_telegram("✅ <b>ทุกอย่างเสร็จสมบูรณ์!</b>\\n\\nโมเดลพร้อมใช้งาน 🎉")